In [1]:
# change to `%matplotlib qt5` for interactive methods
%matplotlib qt5
from pathlib import Path
from imgseries import ImgSeries, ContourTracking, ContourTrackingResults

**NOTE**: the main (numbered) sections are independent of each other

In [2]:
# Define where images are stored, here distributed among two folders
basefolder = Path('data')
folders = [basefolder / folder for folder in ('img1', 'img2')]

# 1) Minimal analysis

## Load images

In [5]:
images = ImgSeries(folders, savepath=basefolder)

# load pre-defined transforms (see ImgSeries for how to define those)
images.load_transforms('for-tests-do-not-modify/Img_Transform')
images

Image Series [extension '.png', folders ['data/img1', 'data/img2'], savepath 'data', 50 files]

## Analysis

In [6]:
# Save results in untracked folder to avoid git tracking test files
ct = ContourTracking(images, savepath='data/untracked_data/')

The line below assumes that contours to follow have already been defined and saved in the metadata file (see details further below)

In [4]:
# Load pre-defined contours, see below how to define them directly
gl.contours.load('../for-tests-do-not-modify/Img_GreyLevel')

In [13]:
ct.contours.show()

<Axes: title={'center': 'img #0, grey level 182'}>

Now, run analysis on these zones.

In [14]:
ct.run()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 197.18it/s]


If using an interactive matplotlib backend, it is also possible to view the analysis in real time (slower). Note that here the parallel option is not available (contrary to GreyLevel), because analysis on different images is not independent.

In [21]:
ct.run(live=True)

Results are stored in the `results.data` attribute, which is a pandas DataFrame (times are automatically extracted from image creation date, but can be modified, see further below)). `x, y` represent position, `p` perimeter and `a` signed area (see **imgbasics** package)

*Note*: if contour detection fails at some point, `data` will contain `NaN` (`numpy.nan`) at the corresponding locations.

*Note*: by default, the full coordinates of the contours are also stored in `results.raw_contour_data`. If this is too heavey, `ContourTracking` can be instanciated with the option `save_raw_contours=False`

In [16]:
ct.results.data.tail()

,folder,filename,time (unix),x1,y1,p1,a1,x2,y2,p2,a2,x3,y3,p3,a3
num,,,,,,,,,,,,,,,
45,img2,img-00655.png,1.696408e+09,186.466511,268.658266,44.788192,-151.820974,258.889911,269.580909,52.257163,-209.578768,322.767246,234.031024,47.293255,-174.203633
46,img2,img-00656.png,1.696408e+09,186.421866,268.7626,44.82619,-152.199486,259.041463,269.671973,52.275865,-210.244179,323.066706,234.038337,47.427696,-175.355538
47,img2,img-00657.png,1.696408e+09,186.451315,268.945876,44.93506,-152.879033,259.271238,269.846878,52.450949,-211.681457,323.456939,234.148139,47.593884,-176.552305
48,img2,img-00658.png,1.696408e+09,186.345269,269.088712,45.135705,-154.196262,259.348315,269.973242,52.534686,-212.449979,323.705796,234.208636,47.719038,-177.653617
49,img2,img-00659.png,1.696408e+09,186.248449,269.262724,45.322188,-155.137694,259.45787,270.162568,52.581465,-212.623299,323.986771,234.329816,47.842566,-178.63742


In [17]:
ct.results.raw_contour_data['1']['10']['y'][:10]

[274.0238095238095,
 274.0,
 273.984375,
 273.8939393939394,
 273.80263157894734,
 273.6981132075472,
 273.45454545454544,
 273.125,
 273.0,
 272.3076923076923]

In [18]:
ct.inspect(skip=3)

In [23]:
ct.animate(start=10, end=30)

Plot perimeters of all detected particles (contours) as a function of time:

In [19]:
ct.results.data.set_index('time (unix)').filter(like='p').plot()

<Axes: xlabel='time (unix)'>

Save data in a tsv (tab-separated value) file, using default filename (filename can be set as a parameter if necessary, see further below). Metadata including contour info, path info, and code version info is also saved in a .json file at the same time. Full (raw) contour data is also saved if required.

Before saving, make sure the timing info is correct for all images. If not, correct timing info, re-run the analysis, and call `results.save()`.

In [20]:
ct.results.save()

# Live view of analysis

In [26]:
ct.run(end=30, live=True)

It is possible that the live animation is non blocking and thus that data is not saved correctly (saving is done at the beginning of the animation and thus data is empty). If this happens, best is probably to re-run the analysis with live=False.

Check that data is empty:

In [28]:
ct.results.data.set_index('time (unix)').filter(like='p').plot()

<Axes: xlabel='time (unix)'>

# 2) Defining and viewing contours

In [3]:
images = ImgSeries(folders, savepath=basefolder)
images.load_transforms('for-tests-do-not-modify/Img_Transform')
ct = ContourTracking(images, savepath='data/untracked_data/')

Defining contours has to be done at least once.

**Important**: Matplotlib must be in an interactive mode to do so.

Defining does not need to be done again in the following situations:
- calling methods again from the same `ct` object, e.g. `ct.run()`
- calling `ct.contours.load()` or `ct.regenerate()` to load contours data from saved metadata (.json) file.

**Note**: to improve contour detection, it might be necessary to do additional cropping/filtering on images with `images.crop` and `images.filter`

In [4]:
ct.threshold.define()

In [5]:
print(ct.threshold)

Threshold object {'value': 182}


In [6]:
ct.contours.define()  # define one contour on the first image of the series

In [10]:
ct.contours.define(n=3)  # define 3 contours on the first image of the series

In [6]:
ct.contours.define(3, num=10)  # define 3 contours at level 170 on image #10 in the series

Viewing analysis zones after defining or loading them:

In [7]:
ct.contours.data

{'position': {'contour 1': (188.04110067355455, 267.5245090373116),
  'contour 2': (258.9021791530991, 268.53190375527873),
  'contour 3': (321.2973382438927, 233.5584805632059)},
 'level': 182,
 'image': 10}

In [12]:
ct.contours.show()  # show contours on the image they have been defined on

<Axes: title={'center': 'img #0, grey level 182'}>

In [13]:
ct.threshold.load()

In [14]:
ct.threshold

Threshold object {'value': 182}

**Note**:
Before, saving contour data was only done by calling `ct.save()`, which saves both data and metadata.
Now, preliminary saving of zone data can be done with `ct.contours.save()`; 
Note that `ct.save()` overwrites that data if the same filename is provided.

In [16]:
ct.contours.save()

# 3) Analyze only subset of images

See **Examples_GreyLevels.ipynb**, and replace:
- `GreyLevel` →  `ContourTracking`
- `zones` →  `contours`

# 4) Load analysis data a posteriori

## Load results without images

In [35]:
# Here, the savepath is where the results have been stored with save()
results = ContourTrackingResults(savepath='data/for-tests-do-not-modify/')

In [36]:
results.load()
results.data.head()

,folder,filename,time (unix),x1,y1,p1,a1,x2,y2,p2,a2,x3,y3,p3,a3
num,,,,,,,,,,,,,,,
0,img1,img-00610.png,1.696408e+09,186.381073,268.880349,44.742100,-151.322772,258.937560,269.789442,52.311205,-210.492459,322.935738,234.149331,47.418369,-175.188267
1,img1,img-00611.png,1.696408e+09,186.465505,268.617503,44.690003,-150.966001,258.831991,269.534000,52.257060,-209.595277,322.669741,233.992961,47.321906,-173.954602
2,img1,img-00612.png,1.696408e+09,186.533386,268.466188,44.553489,-149.860104,258.721319,269.418015,52.161628,-208.960533,322.396355,233.933190,47.303397,-173.368130
3,img1,img-00613.png,1.696408e+09,186.565485,268.256760,44.416100,-148.986836,258.573698,269.210823,52.127305,-208.499205,322.083884,233.802949,47.195826,-172.385620
4,img1,img-00614.png,1.696408e+09,186.588163,268.064450,44.148188,-147.764186,258.448329,269.019657,52.096051,-207.787577,321.791162,233.689884,47.079396,-171.058334


In [37]:
results.metadata

{'path': ['/Users/olivier.vincent/Python-OV/imgseries/data/untracked_data'],
 'folders': ['../img1', '../img2'],
 'contours': {'position': {'contour 1': [186.38107291551614,
    268.8803487163035],
   'contour 2': [258.93756035280825, 269.7894418257783],
   'contour 3': [322.9357384733174, 234.14933143477805]},
  'level': 182,
  'image': 0},
 'grayscale': {},
 'rotation': {'angle': 22.765009107511496},
 'crop': {'zone': [159, 171, 467, 380]},
 'filter': {},
 'subtraction': {},
 'threshold': {},
 'time (utc)': '2024-02-19 11:38:36',
 'code version': {'skimage': {'status': 'not a git repository',
   'tag': 'v0.19.3'},
  'imgseries': {'hash': '9cd9673f60328c205afe3ac5fcad64c7124e0982',
   'status': 'dirty'},
  'imgbasics': {'status': 'not a git repository', 'tag': 'v0.3.0'},
  'filo': {'hash': '150574b5ae82c74d0c500b3fdb494e8f9c7631f7',
   'status': 'clean',
   'tag': 'v1.1.5'},
  'matplotlib': {'status': 'not a git repository', 'tag': 'v3.7.0'},
  'numpy': {'status': 'not a git repositor

In [38]:
results.raw_contour_data['2']['33']['x'][:10]

[258.0,
 257.8333333333333,
 257.0,
 256.0,
 255.0,
 254.0,
 253.5,
 253.0,
 252.0,
 251.6315789473684]

# 5) Connect results to image series (e.g. for inspection/visualization)

In [39]:
images = ImgSeries(folders, savepath=basefolder)
ct = ContourTracking(images)

# Line below is equivalent to call results.load() and images.load_transforms,
# except that the transforms are taken directly from the results metadata.
ct.regenerate(filename='for-tests-do-not-modify/Img_ContourTracking')

In [40]:
print('Image Crop -------------------------', ct.img_series.crop.zone)
print('Image Crop (same as above) ---------', ct.results.metadata['crop'].get('zone', ()))
print('')
print('Image Rotation ---------------------', ct.img_series.rotation.data)
print('Image Rotation (same as above) -----', ct.results.metadata['rotation'])
print('')
print('Contour position -------------------', ct.contours.data['position']['contour 2'])
print('Contour position (same as above) ---', ct.results.metadata['contours']['position']['contour 2'])

ct.results.data.head()

Image Crop ------------------------- [159, 171, 467, 380]
Image Crop (same as above) --------- [159, 171, 467, 380]

Image Rotation --------------------- {'angle': 22.765009107511496}
Image Rotation (same as above) ----- {'angle': 22.765009107511496}

Contour position ------------------- [258.93756035280825, 269.7894418257783]
Contour position (same as above) --- [258.93756035280825, 269.7894418257783]


,folder,filename,time (unix),x1,y1,p1,a1,x2,y2,p2,a2,x3,y3,p3,a3
num,,,,,,,,,,,,,,,
0,img1,img-00610.png,1.696408e+09,186.381073,268.880349,44.742100,-151.322772,258.937560,269.789442,52.311205,-210.492459,322.935738,234.149331,47.418369,-175.188267
1,img1,img-00611.png,1.696408e+09,186.465505,268.617503,44.690003,-150.966001,258.831991,269.534000,52.257060,-209.595277,322.669741,233.992961,47.321906,-173.954602
2,img1,img-00612.png,1.696408e+09,186.533386,268.466188,44.553489,-149.860104,258.721319,269.418015,52.161628,-208.960533,322.396355,233.933190,47.303397,-173.368130
3,img1,img-00613.png,1.696408e+09,186.565485,268.256760,44.416100,-148.986836,258.573698,269.210823,52.127305,-208.499205,322.083884,233.802949,47.195826,-172.385620
4,img1,img-00614.png,1.696408e+09,186.588163,268.064450,44.148188,-147.764186,258.448329,269.019657,52.096051,-207.787577,321.791162,233.689884,47.079396,-171.058334


Once the analysis is regenerated, all the tools associated with images (inspection, showing, animation, etc.) are available:

In [33]:
ct.show(num=25)

(<Axes: title={'center': 'img #25, grey level None'}>,)

In [34]:
ct.inspect()